<h2>데이터 탐색</h2>

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, f1_score
from sklearn import model_selection

from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor

from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# submit csv 만드는 코드
submit = pd.read_csv('sample_submission.csv')
submit['Class'] = y_pred
submit.to_csv('submit.csv', index=False)

In [2]:
# 데이터 load 
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')

# shape 확인
train.shape, val.shape, test.shape

((113842, 31), (28462, 32), (142503, 31))

In [3]:
# 검증 데이터를 통해 타겟 비율 확인
val_normal, val_fraud = val['Class'].value_counts()
val_contamination = val_fraud / val_normal
print(f'정상 데이터 : {val_normal}, 사기 데이터 : {val_fraud}')
print(f'검증 데이터의 사기 비율 : [{val_contamination}]')

정상 데이터 : 28432, 사기 데이터 : 30
검증 데이터의 사기 비율 : [0.0010551491277433877]


In [4]:
# 데이터 셋 정리
X_train = train.drop('ID', axis=1)

X_val = val.drop(['ID', 'Class'], axis=1)
y_val = val['Class']

X_test = test.drop('ID', axis=1)

# shape 확인
X_train.shape, X_val.shape, y_val.shape, X_test.shape

((113842, 30), (28462, 30), (28462,), (142503, 30))

<hr>

<h2>IsolationForest(Base Code 참조)</h2>

In [5]:
# 모델 출력값 변환 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
def get_pred_label(model_pred):
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [6]:
# 모델 학습
IF = IsolationForest(contamination=val_contamination, random_state=42)
IF.fit(X_train)
IF_val_pred = IF.predict(X_val)
IF_val_pred = get_pred_label(IF_val_pred)

# 모델 결과 확인
unique, counts = np.unique(IF_val_pred, return_counts=True)
IF_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{IF_contamination}]')

print(classification_report(y_val, IF_val_pred))

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


클래스 : [0 1], 클래스별 개수 : [28435    27]
정상 데이터 : 28435, 사기 데이터 : 27
검증 데이터의 사기 비율 : [0.0009495340249692281]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.37      0.33      0.35        30

    accuracy                           1.00     28462
   macro avg       0.68      0.67      0.68     28462
weighted avg       1.00      1.00      1.00     28462



<h2>K-Means</h2>

In [49]:
# 모델 학습
kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=300, random_state=42)
kmeans.fit(X_train)
kmeans_val_pred = kmeans.predict(X_val)

# 모델 결과 확인
unique, counts = np.unique(kmeans_val_pred, return_counts=True)
kmeans_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{kmeans_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, kmeans_val_pred))

클래스 : [0 1], 클래스별 개수 : [27911   551]
정상 데이터 : 27911, 사기 데이터 : 551
검증 데이터의 사기 비율 : [0.019741320626276378]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     28432
           1       0.00      0.00      0.00        30

    accuracy                           0.98     28462
   macro avg       0.50      0.49      0.49     28462
weighted avg       1.00      0.98      0.99     28462



<h2>MeanShift</h2>

In [ ]:
# 모델 학습
MS = MeanShift(bandwidth=2)
MS.fit(X_train)
MS_val_pred = MS.predict(X_val)

# 모델 결과 확인
unique, counts = np.unique(MS_val_pred, return_counts=True)
MS_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{MS_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, MS_val_pred))

<h2>GMM</h2>

In [52]:
# 모델 학습
gmm = GaussianMixture(n_components=2, random_state=42)
gmm.fit(X_train)
gmm_val_pred = gmm.predict(X_val)

# 모델 결과 확인
unique, counts = np.unique(gmm_val_pred, return_counts=True)
gmm_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{gmm_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, gmm_val_pred))

클래스 : [0 1], 클래스별 개수 : [17872 10590]
정상 데이터 : 17872, 사기 데이터 : 10590
검증 데이터의 사기 비율 : [0.5925470008952551]
              precision    recall  f1-score   support

           0       1.00      0.63      0.77     28432
           1       0.00      0.90      0.01        30

    accuracy                           0.63     28462
   macro avg       0.50      0.76      0.39     28462
weighted avg       1.00      0.63      0.77     28462



<h2>DBSCAN</h2>

In [33]:
def get_pred_label_db(model_pred):
    model_pred = np.where(model_pred == -1, 0, model_pred)
    model_pred = np.where(model_pred != 0, 1, model_pred)
    return model_pred

In [35]:
# 모델 학습
dbscan = DBSCAN(metric='euclidean')
dbscan_val_pred = dbscan.fit_predict(X_val)
dbscan_val_pred = get_pred_label_db(dbscan_val_pred)

# 모델 결과 확인
unique, counts = np.unique(dbscan_val_pred, return_counts=True)
dbscan_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{dbscan_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, dbscan_val_pred))

클래스 : [0 1], 클래스별 개수 : [26246  2216]
정상 데이터 : 26246, 사기 데이터 : 2216
검증 데이터의 사기 비율 : [0.0844319134344281]
              precision    recall  f1-score   support

           0       1.00      0.92      0.96     28432
           1       0.00      0.00      0.00        30

    accuracy                           0.92     28462
   macro avg       0.50      0.46      0.48     28462
weighted avg       1.00      0.92      0.96     28462



<h2>One-Class SVM</h2>

In [11]:
# 모델 학습
ocsvm=OneClassSVM()
ocsvm.fit(X_train)
ocsvm_val_pred = ocsvm.predict(X_val)
ocsvm_val_pred = get_pred_label(ocsvm_val_pred)

# 모델 결과 확인
unique, counts = np.unique(ocsvm_val_pred, return_counts=True)
unique, counts
ocsvm_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{ocsvm_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, ocsvm_val_pred))

클래스 : [0 1], 클래스별 개수 : [14338 14124]
정상 데이터 : 14338, 사기 데이터 : 14124
검증 데이터의 사기 비율 : [0.9850746268656716]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67     28432
           1       0.00      1.00      0.00        30

    accuracy                           0.50     28462
   macro avg       0.50      0.75      0.34     28462
weighted avg       1.00      0.50      0.67     28462



<h2>EllipticEnvelope</h2>

In [6]:
# 모델 학습
ell = EllipticEnvelope(contamination=val_contamination)
ell.fit(X_train)
ell_val_pred = ell.predict(X_val)
ell_val_pred = get_pred_label(ell_val_pred)

# 모델 결과 확인
unique, counts = np.unique(ell_val_pred, return_counts=True)
ell_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{ell_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, ell_val_pred))

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-226.869431363108248 > -273.148825330976365). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-190.782981684247972 > -213.775477031065947). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-196.597074200819691 > -283.710709677103864). You may want to try with a higher value of support_fraction (current val

클래스 : [0 1], 클래스별 개수 : [28431    31]
정상 데이터 : 28431, 사기 데이터 : 31
검증 데이터의 사기 비율 : [0.001090359115050473]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.00      0.00      0.00        30

    accuracy                           1.00     28462
   macro avg       0.50      0.50      0.50     28462
weighted avg       1.00      1.00      1.00     28462



<h2>LOF</h2>

In [7]:
# 모델 학습
lof = LocalOutlierFactor(contamination=val_contamination)
lof_val_pred = lof.fit_predict(X_val)
lof_val_pred = get_pred_label(lof_val_pred)

# 모델 결과 확인
unique, counts = np.unique(lof_val_pred, return_counts=True)
lof_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{lof_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, lof_val_pred))

클래스 : [0 1], 클래스별 개수 : [28431    31]
정상 데이터 : 28431, 사기 데이터 : 31
검증 데이터의 사기 비율 : [0.001090359115050473]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.06      0.07      0.07        30

    accuracy                           1.00     28462
   macro avg       0.53      0.53      0.53     28462
weighted avg       1.00      1.00      1.00     28462



<hr>

<h2>Aoutoencoder</h2>

<h2>Bayesian network</h2>

<h2>Decision Rule</h2>

<hr>

<h2>하이퍼 파라미터 튜닝(RandomizedSearchCV)</h2>

In [9]:
random_state = 42

<h4>IsolationForest</h4>

In [12]:
params = {'n_estimators': list(range(100, 500, 10)), 
          'max_samples': list(range(100, 500, 10)), 
          'contamination': [0.0001, 0.0005, 0.001, 0.002, 0.005], 
          'max_features': range(1, 15), 
          'bootstrap': [True, False]
         }

f1sc = make_scorer(f1_score, average='micro')

IF_random_dt_estimator = model_selection.RandomizedSearchCV(IF, 
                                                        params,
                                                        scoring=f1sc, 
                                                        cv=5, 
                                                        return_train_score=True,
                                                        random_state = random_state,
                                                        n_iter=50,
                                                        n_jobs=-1
                                                        )
IF_random_dt_estimator.fit(X_val, y_val)

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=IsolationForest(contamination=0.0010551491277433877,
                                             random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'contamination': [0.0001, 0.0005, 0.001,
                                                          0.002, 0.005],
                                        'max_features': range(1, 15),
                                        'max_samples': [100, 110, 120, 130, 140,
                                                        150, 160, 170, 180, 190,
                                                        200, 210, 220, 230, 240,
                                                        250, 260, 270, 280, 290,
                                                        300, 310, 320, 330, 340,
                                                        350, 360, 370, 380, 390, ...],
                                        'n_estimators': [100, 110, 120, 130,
                                                         140, 150, 160, 170,
                                                         180, 190, 200, 210,
                                                         220, 230, 240, 250,
                                                         260, 270, 280, 290,
                                                         300, 310, 320, 330,
                                                         340, 350, 360, 370,
                                                         380, 390, ...]},
                   random_state=42, return_train_score=True,
                   scoring=make_scorer(f1_score, average=micro))

In [13]:
print(f'Best score : {IF_random_dt_estimator.best_score_}')
print(f'Best_estimator : {IF_random_dt_estimator.best_estimator_}')

Best score : 0.0010540307973977486
Best_estimator : IsolationForest(bootstrap=True, contamination=0.0001, max_features=11,
                max_samples=100, n_estimators=330, random_state=42)


<h4>EllipticEnvelope</h4>

In [ ]:
params = {'store_precision' : [True, False], 
          'assume_centered' : [True, False], 
          'support_fraction' : range(0, 1),
          'contamination': [0.0001, 0.0005, 0.001, 0.002, 0.005]
         }

f1sc = make_scorer(f1_score, average='micro')

ell_random_dt_estimator = model_selection.RandomizedSearchCV(ell, 
                                                        params,
                                                        scoring=f1sc, 
                                                        cv=3, 
                                                        return_train_score=True,
                                                        random_state = random_state,
                                                        n_iter=10,
                                                        n_jobs=-1
                                                        )
ell_random_dt_estimator.fit(X_val, y_val)

In [ ]:
print(f'Best score : {ell_random_dt_estimator.best_score_}')
print(f'Best_estimator : {ell_random_dt_estimator.best_estimator_}')

<h4>LOF</h4>

In [12]:
params = {'n_neighbors': list(range(10, 50, 10)), 
          'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
          'leaf_size' : list(range(10, 30, 10)), 
          'p' : range(2, 10)
         }

f1sc = make_scorer(f1_score, average='micro')

lof_random_dt_estimator = model_selection.RandomizedSearchCV(lof, 
                                                        params,
                                                        scoring=f1sc, 
                                                        cv=3, 
                                                        return_train_score=True,
                                                        random_state = random_state,
                                                        n_iter=10,
                                                        n_jobs=-1
                                                        )
lof_random_dt_estimator.fit(X_val, y_val)

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the train scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


RandomizedSearchCV(cv=3,
                   estimator=LocalOutlierFactor(contamination=0.0010551491277433877),
                   n_jobs=-1,
                   param_distributions={'algorithm': ['ball_tree', 'kd_tree',
                                                      'brute'],
                                        'leaf_size': [10, 20],
                                        'n_neighbors': [10, 20, 30, 40],
                                        'p': range(2, 10)},
                   random_state=42, return_train_score=True,
                   scoring=make_scorer(f1_score, average=micro))

In [13]:
print(f'Best score : {lof_random_dt_estimator.best_score_}')
print(f'Best_estimator : {lof_random_dt_estimator.best_estimator_}')

Best score : nan
Best_estimator : LocalOutlierFactor(algorithm='ball_tree', contamination=0.0010551491277433877,
                   leaf_size=20, p=7)


<hr>

<h4>최적의 파라미터로 튜닝 후 학습 성능 확인</h4>

In [17]:
# IF 모델 튜닝 전

# 모델 학습
IF = IsolationForest(contamination=val_contamination, random_state=42)
IF.fit(X_train)
IF_val_pred = IF.predict(X_val)
IF_val_pred = get_pred_label(IF_val_pred)

# 모델 결과 확인
unique, counts = np.unique(IF_val_pred, return_counts=True)
IF_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{IF_contamination}]')

print(classification_report(y_val, IF_val_pred))

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


클래스 : [0 1], 클래스별 개수 : [28435    27]
정상 데이터 : 28435, 사기 데이터 : 27
검증 데이터의 사기 비율 : [0.0009495340249692281]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.37      0.33      0.35        30

    accuracy                           1.00     28462
   macro avg       0.68      0.67      0.68     28462
weighted avg       1.00      1.00      1.00     28462



In [18]:
# IF 모델 튜닝 후

# 모델 학습
IF = IsolationForest(contamination=val_contamination, 
                     bootstrap=True, 
                     max_features=11,
                     max_samples=100, 
                     n_estimators=330,
                     random_state=42)
IF.fit(X_train)
IF_val_pred = IF.predict(X_val)
IF_val_pred = get_pred_label(IF_val_pred)

# 모델 결과 확인
unique, counts = np.unique(IF_val_pred, return_counts=True)
IF_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{IF_contamination}]')

print(classification_report(y_val, IF_val_pred))

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


클래스 : [0 1], 클래스별 개수 : [28439    23]
정상 데이터 : 28439, 사기 데이터 : 23
검증 데이터의 사기 비율 : [0.000808748549527058]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.52      0.40      0.45        30

    accuracy                           1.00     28462
   macro avg       0.76      0.70      0.73     28462
weighted avg       1.00      1.00      1.00     28462



In [ ]:
# ell 모델 튜닝 전

# 모델 학습
ell = EllipticEnvelope(contamination=val_contamination)
ell.fit(X_train)
ell_val_pred = ell.predict(X_val)
ell_val_pred = get_pred_label(ell_val_pred)

# 모델 결과 확인
unique, counts = np.unique(ell_val_pred, return_counts=True)
ell_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{ell_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, ell_val_pred))

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-252.752532577695092 > -252.851603765807965). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-239.051360704405170 > -247.258843767835373). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-153.658014211652528 > -267.199030985352294). You may want to try with a higher value of support_fraction (current val

클래스 : [0 1], 클래스별 개수 : [28431    31]
정상 데이터 : 28431, 사기 데이터 : 31
검증 데이터의 사기 비율 : [0.001090359115050473]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.00      0.00      0.00        30

    accuracy                           1.00     28462
   macro avg       0.50      0.50      0.50     28462
weighted avg       1.00      1.00      1.00     28462



In [19]:
# ell 모델 튜닝 후

# 모델 학습
ell = EllipticEnvelope(store_precision=True,
                       assume_centered = True,
                       support_fraction = 0.5,
                       contamination=val_contamination)
ell.fit(X_train)
ell_val_pred = ell.predict(X_val)
ell_val_pred = get_pred_label(ell_val_pred)

# 모델 결과 확인
unique, counts = np.unique(ell_val_pred, return_counts=True)
ell_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{ell_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, ell_val_pred))

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-215.406297867087346 > -248.625132348445106). You may want to try with a higher value of support_fraction (current value: 0.500).
  warnings.warn(
c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-254.876344246054970 > -284.319447897467683). You may want to try with a higher value of support_fraction (current value: 0.500).
  warnings.warn(
c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-187.416101514658578 > -275.069312612828242). You may want to try with a higher value of support_fraction (current val

클래스 : [0 1], 클래스별 개수 : [28439    23]
정상 데이터 : 28439, 사기 데이터 : 23
검증 데이터의 사기 비율 : [0.000808748549527058]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.00      0.00      0.00        30

    accuracy                           1.00     28462
   macro avg       0.50      0.50      0.50     28462
weighted avg       1.00      1.00      1.00     28462



In [ ]:
# lof 모델 튜닝 전

# 모델 학습
lof = LocalOutlierFactor(contamination=val_contamination)
lof_val_pred = lof.fit_predict(X_val)
lof_val_pred = get_pred_label(lof_val_pred)

# 모델 결과 확인
unique, counts = np.unique(lof_val_pred, return_counts=True)
lof_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{lof_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, lof_val_pred))

클래스 : [0 1], 클래스별 개수 : [28431    31]
정상 데이터 : 28431, 사기 데이터 : 31
검증 데이터의 사기 비율 : [0.001090359115050473]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.06      0.07      0.07        30

    accuracy                           1.00     28462
   macro avg       0.53      0.53      0.53     28462
weighted avg       1.00      1.00      1.00     28462



In [20]:
# lof 모델 튜닝 후

# 모델 학습
lof = LocalOutlierFactor(contamination=val_contamination,
                         algorithm='ball_tree', 
                         leaf_size=20, 
                         p=7)
lof_val_pred = lof.fit_predict(X_val)
lof_val_pred = get_pred_label(lof_val_pred)

# 모델 결과 확인
unique, counts = np.unique(lof_val_pred, return_counts=True)
lof_contamination = counts[1] / counts[0]
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{lof_contamination}]')

# 모델 성능 확인
print(classification_report(y_val, lof_val_pred))

클래스 : [0 1], 클래스별 개수 : [28431    31]
정상 데이터 : 28431, 사기 데이터 : 31
검증 데이터의 사기 비율 : [0.001090359115050473]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.26      0.27      0.26        30

    accuracy                           1.00     28462
   macro avg       0.63      0.63      0.63     28462
weighted avg       1.00      1.00      1.00     28462



<hr>

<h2>파라미터 튜닝 후 결과 제출</h2>

In [21]:
# test 데이터로 label 만들기
IF_test_pred = IF.fit_predict(X_test)
IF_test_pred = get_pred_label(IF_test_pred)

# submit csv 만드는 코드
submit_IF = pd.read_csv('sample_submission.csv')
submit_IF['Class'] = IF_test_pred
submit_IF.to_csv('submit_IF.csv', index=False)

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [22]:
# test 데이터로 label 만들기
ell_test_pred = ell.fit_predict(X_test)
ell_test_pred = get_pred_label(ell_test_pred)

# submit csv 만드는 코드
submit_ell = pd.read_csv('sample_submission.csv')
submit_ell['Class'] = ell_test_pred
submit_ell.to_csv('submit_ell.csv', index=False)

c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-243.658142871325737 > -310.482580930041593). You may want to try with a higher value of support_fraction (current value: 0.500).
  warnings.warn(
c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-199.169208252394924 > -199.357312950742312). You may want to try with a higher value of support_fraction (current value: 0.500).
  warnings.warn(
c:\ProgramData\Anaconda3\envs\card\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-256.608441588840719 > -257.681488982745293). You may want to try with a higher value of support_fraction (current val

In [23]:
# test 데이터로 label 만들기
lof_test_pred = lof.fit_predict(X_test)
lof_test_pred = get_pred_label(lof_test_pred)

# submit csv 만드는 코드
submit_lof = pd.read_csv('sample_submission.csv')
submit_lof['Class'] = lof_test_pred
submit_lof.to_csv('submit_lof.csv', index=False)